## 2001UNHS-0041 WGS

### February 7, 2020

### notebook for processing the UNHS WGS data
#### based on script from Raph Gibbs (CBG)

#### global varibles and libraries

In [ ]:
#global notebook variables for both python and bash majic (by stdin arguments)
# WRKDIR = '/Users/mooreank/Desktop/WGS'
# PRJ_BUCKET = 'gs://nihnialng-pd-wgs'
# PROJECT_ID = 'pd-genome'
# MYUSER = 'mooreank'
# AUTOSOMES=[str(x) for x in list(range(1,23))]
# SEXOMES=['X','Y']
# CHROMOSOMES=AUTOSOMES + SEXOMES
# COHORT='ninds_eopd'
# COHORTBUILD='{}.july2019'.format(COHORT)
# COHORT_BUCKET='{}/{}'.format(PRJ_BUCKET,COHORT)

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import time
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
WRKDIR = '/labshare/anni/UNHS'

#### see what fastqs are present in the stagin bucket

In [ ]:

#staging bucket that USUHS uploaded fastqs to
!ls /labshare/anni/UNHS/2001UNHS-0041/ | wc -l
# ls gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.01 | wc -l
    
# ls gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.00 | head
# ls gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.01 | head

In [ ]:
dest_bucket_path = '{}/{}/fastqs'.format(PRJ_BUCKET,COHORT)
gcs_fastq_mv_cmd = 'gsutil -mq cp gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.0*/*.fastq.gz \
{}/'.format(dest_bucket_path)

print('#run these commands at terminal:\n')
print('#WE\'VE ALREADY RUN THIS SO DO NOT NEED TO COPY AGAIN\n')
print(gcs_fastq_mv_cmd)

In [ ]:
%%bash -s "$dest_bucket_path"
#check all the fastqs have been moved should be 592 pairs
DEST_BUCKET=${1}

gsutil ls ${DEST_BUCKET}/*_R1_001.fastq.gz | wc -l
gsutil ls ${DEST_BUCKET}/*_R2_001.fastq.gz | wc -l

#### get fastqs listing

In [ ]:
%%bash -s "$WRKDIR" "$dest_bucket_path" "$COHORT"
#get fastq listing and tokenize
WRKDIR=${1}
DEST_BUCKET=${2}
COHORT=${3}

FASTQ_LISTING=${WRKDIR}/${COHORT}.fastq.listing.txt

gsutil ls ${DEST_BUCKET}/*.fastq.gz > ${FASTQ_LISTING}

sed -i s"/gs:\/\/nihnialng-pd-wgs\/${COHORT}\/fastqs\///"g ${FASTQ_LISTING}
sed -i s"/\.fastq\.gz//"g ${FASTQ_LISTING}
sed -i s"/_/\\t/"g ${FASTQ_LISTING}

less ${FASTQ_LISTING} | wc -l
head ${FASTQ_LISTING}
tail ${FASTQ_LISTING}

#### load fastq listing and USUHS sample info and ID mappings

In [ ]:
#ok now see what we have
#load the USUHS quality reports
usuhs_qa_file = WRKDIR + '/pA3.QA.june.26.2019.xlsx'
usuhs_qa = pd.read_excel(usuhs_qa_file)
print(usuhs_qa.shape)

#load the fastq info
fastqs_file = '{}/{}.fastq.listing.txt'.format(WRKDIR,COHORT)
fastqs_df = pd.read_csv(fastqs_file,header=None,sep='\t')
fastqs_df.columns = ['usuhsID','S','LANE','READ','NUM']
print(fastqs_df.shape)

#load usuhs key map
usuhs_key_file = WRKDIR + '/pA3.hernandez.export1.keyTable.june.27.2019.csv'
usuhs_keys_df = pd.read_csv(usuhs_key_file,sep=',')
print(usuhs_keys_df.shape)


In [ ]:
print(usuhs_keys_df.shape)
usuhs_keys_df.head()

#### unfortunately of 'real' sampleID is packed in the middle of the 'Description' column values, extract it

In [ ]:
usuhs_keys_df['grbg1'],usuhs_keys_df['grbg2'],usuhs_keys_df['oriID'],usuhs_keys_df['grbg3'] = \
usuhs_keys_df['Description'].str.split('-').str
usuhs_keys_df.drop(['grbg1','grbg2','grbg3'],axis='columns',inplace=True)

In [ ]:
#take a look at the USUHS QA table, if desired
usuhs_qa.head()

In [ ]:
#counts look right but check to see if any not found id
print(set(fastqs_df['usuhsID']) - set(usuhs_keys_df['SampleID']))
print(set(usuhs_keys_df['SampleID']) - set(fastqs_df['usuhsID']))

In [ ]:
#merge the fastqs and the key maps
named_fastqs_df = pd.merge(fastqs_df,usuhs_keys_df,how='left',left_on='usuhsID',\
                           right_on='SampleID')

In [ ]:
print(fastqs_df.shape)
print(named_fastqs_df.shape)
named_fastqs_df.head()

In [ ]:
#the NUM bit of the file name if always '001' so format the field
print(named_fastqs_df['NUM'].value_counts())
named_fastqs_df['NUM'] = '001'

In [ ]:
#check exprect count R1 == R2
named_fastqs_df['READ'].value_counts()

In [ ]:
#only need one row for each sample, not both fastq pairs, ie R1 and R2
cohort_df = named_fastqs_df.loc[named_fastqs_df['READ'] == 'R1']
print(cohort_df.shape)
cohort_df.head()

#### format per sample jsons files for the fastq to ubam jobs

In [ ]:
def checkfortemplatefile(this_template_file):
    if not os.path.isfile(this_template_file):
        print('need ' + this_template_file)

In [ ]:
#create the jsons directory for the fastq to bam if it doesn't exist
#also check to see if the blank jsons are present or you need to retrieve
fastq_to_bam_json_dir = '{}/jsons'.format(WRKDIR)

if os.path.isdir(fastq_to_bam_json_dir):
    os.makedirs(fastq_to_bam_json_dir + '/fastqtoubam', exist_ok=True)    
    
checkfortemplatefile(fastq_to_bam_json_dir + '/blank.fastqtoubam.json')
checkfortemplatefile(fastq_to_bam_json_dir + '/blank.broadbam.hg38.json')
checkfortemplatefile(fastq_to_bam_json_dir + '/blank.align.label.json')

#### here we are going to subet to just a 3rd or the sample, so each of you have a subset to process

so you have to save you subset list to the file path below

In [ ]:
your_subset_file = '{}/anni_subset_of_full_cohort.list'.format(WRKDIR)
your_subset = pd.read_csv(your_subset_file,header=None)
print(your_subset.shape)

print(cohort_df.shape)
cohort_df = cohort_df.loc[cohort_df['oriID'].isin(your_subset[0])]
print(cohort_df.shape)
cohort_df.head()

In [ ]:
#format the WF input jsons for fastq to ubams
from datetime import datetime

json_template = '{}/jsons/blank.fastqtoubam.json'.format(WRKDIR)

fastq_format = '{}/{}/fastqs/{}_{}_{}_{}_001.fastq.gz'

DEFAULTATTEMPTS = 3
DEFAULTDISK = 500
DEFAULTMEM = '30 GB'

sample_ids = cohort_df['oriID'].unique()
for sample_id in sample_ids:
    with open(json_template) as json_file:  
        data = json.load(json_file)
        data['ConvertPairedFastQsToUnmappedBamWf.PairedFastQsToUnmappedBAM.gatk_path'] = '/gatk/gatk'
        data['ConvertPairedFastQsToUnmappedBamWf.PairedFastQsToUnmappedBAM.docker'] = 'broadinstitute/gatk:4.0.1.2'
        data['ConvertPairedFastQsToUnmappedBamWf.PairedFastQsToUnmappedBAM.mem_size'] = DEFAULTMEM
        data['ConvertPairedFastQsToUnmappedBamWf.PairedFastQsToUnmappedBAM.disk_size'] = DEFAULTDISK
        data['ConvertPairedFastQsToUnmappedBamWf.PairedFastQsToUnmappedBAM.preemptible_tries'] = DEFAULTATTEMPTS

        sample_data = data.copy()
        sample_data['ConvertPairedFastQsToUnmappedBamWf.readgroup_list'] = []
        sample_data['ConvertPairedFastQsToUnmappedBamWf.metadata'] = {}
        sample_data['ConvertPairedFastQsToUnmappedBamWf.fastq_pairs'] = {}
        sample_df = cohort_df.loc[cohort_df['oriID'] == sample_id]
        json_outfile_name = '{}/jsons/fastqtoubam/{}.fastqtoubam.json'.format(WRKDIR,sample_id)

        for index, row in sample_df.iterrows():
            lib_date, lib_machine, lib_index, flowcell = row['Flowcell'].split('_')

            read_group_name = '{}_{}_{}_{}'.format(row['oriID'], row['usuhsID'], row['LANE'], lib_date)

            this_date_str = datetime.strptime(lib_date, '%y%m%d').strftime('%Y-%m-%d')
            read_group_info = [row['oriID'], row['oriID'], row['Flowcell'], this_date_str, 'ILLUMINA', 'USUHS']

            read_group_fastqs = [fastq_format.format(PRJ_BUCKET,COHORT,row['usuhsID'],row['S'],row['LANE'],'R1'), \
                                fastq_format.format(PRJ_BUCKET,COHORT,row['usuhsID'],row['S'],row['LANE'],'R2')]

            sample_data['ConvertPairedFastQsToUnmappedBamWf.readgroup_list'].append(read_group_name)
            sample_data['ConvertPairedFastQsToUnmappedBamWf.metadata'].update({read_group_name : read_group_info})
            sample_data['ConvertPairedFastQsToUnmappedBamWf.fastq_pairs'].update({read_group_name : read_group_fastqs})
        
        with open(json_outfile_name,'w') as json_outfile:
            json.dump(sample_data,json_outfile,sort_keys=True,indent=4)

cohort_file_list = '{}/{}.samples.list'.format(WRKDIR,COHORT)
pd.DataFrame(data=sample_ids).to_csv(cohort_file_list,header=False,index=False)

In [ ]:
#define format function the ggp cmd
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'echo -n {SAMPLE} OPID=\n\
gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {PRJ_BUCKET}/resources/tools/wdl_pipeline.yaml \
--zones us-central1-f \
--memory 7 \
--logging {COHORT_BUCKET}/logs/ubams/{SAMPLE} \
--inputs-from-file WDL={WRKDIR}/tools/broad/paired-fastq-to-unmapped-bam.wdl \
--inputs-from-file WORKFLOW_INPUTS={WRKDIR}/jsons/{COHORT}/fastqtoubam/{SAMPLE}.fastqtoubam.json \
--inputs-from-file WORKFLOW_OPTIONS={WRKDIR}/jsons/generic.google-papi.options.json \
--inputs WORKSPACE={COHORT_BUCKET}/workspace/{SAMPLE} \
--inputs OUTPUTS={COHORT_BUCKET}/ubams/{SAMPLE} \
--labels=pipe=fastq_to_ubam,sample={LABELNAME},cohort={LCCOHORT},user={MYUSER}'
    return(this_cmd.format(SAMPLE=this_sample,PROJECT_ID=PROJECT_ID,PRJ_BUCKET=PRJ_BUCKET,\
                         COHORT_BUCKET=chrt_bucket,WRKDIR=WRKDIR,COHORT=COHORT,\
                          LABELNAME=this_sample.lower(),LCCOHORT=COHORT.lower(),MYUSER=MYUSER))

#iterate over samples formatting the cmds
cohort_bucket = '{}/{}'.format(PRJ_BUCKET,COHORT)
cmds = [formatgcpcmd(sample_id,cohort_bucket) for sample_id in sample_ids]

temp_script_file = '{}/{}.run_fastqs_to_ubams.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.run_fastqs_to_ubams.log &'.format(WRKDIR,COHORT.lower()))

In [ ]:
%%bash -s "$PROJECT_ID" "$MYUSER" "$COHORT"
#see if there are instances running the job
PROJECT_ID=${1}
MYUSER=${2}
COHORT=${3}

PIPELABEL=fastq_to_ubam

echo "full job worker node count"
gcloud compute instances list --project ${PROJECT_ID} \
    --filter "labels.pipe=${PIPELABEL} labels.cohort=${COHORT} labels.user=${MYUSER}" | grep RUNNING | wc -l
#echo "job managers"
#gcloud compute instances list --project ${PROJECT_ID} \
#    --filter "labels.pipe=${PIPELABEL} labels.cohort=${COHORT} labels.user=${MYUSER}"

In [ ]:
%%bash

OPID=EI6syK2uLRjvoZHbvPnIuRgglfil2O0VKg9wcm9kdWN0aW9uUXVldWU
gcloud alpha genomics operations describe ${OPID} \
--format='yaml(done, error, metadata.events)'

In [ ]:
%%bash -s "$WRKDIR" "$PRJ_BUCKET" "$COHORT"
#get a list of files that were successfully created
WRKDIR=${1}
PRJ_BUCKET=${2}
COHORT=${3}

COHORT_BUCKET=${PRJ_BUCKET}/${COHORT}

gsutil -mq ls ${COHORT_BUCKET}/ubams > ${WRKDIR}/${COHORT}.found.files

sed -i s"/gs:\/\/nihnialng-pd-wgs\/${COHORT}\/ubams\///"g ${WRKDIR}/${COHORT}.found.files
sed -i s"/\///"g ${WRKDIR}/${COHORT}.found.files

less ${WRKDIR}/${COHORT}.found.files | wc -l

In [ ]:
#check for any missing expected bams
expected_file = '{}/{}.samples.list'.format(WRKDIR, COHORT)
observed_file = '{}/{}.found.files'.format(WRKDIR, COHORT) 
missing_file = '{}/{}.missing.samples.list'.format(WRKDIR, COHORT)

expected = pd.read_csv(expected_file,header=None)
observed = pd.read_csv(observed_file,header=None)

print(expected.shape)
print(observed.shape)

print(len(set(expected[0]) - set(observed[0])))

missing = expected.loc[~expected[0].isin(observed[0])]
print(missing.shape)
missing.head()

#save the missing list
missing.to_csv(missing_file,header=None,index=None)

## THIS IS WHERE WE STOPPED AFTER FASTQ TO uBAM coversion

#### clean up the temp workspace and logs from the fastq to ubam conversions

In [ ]:
print('#run these commands at terminal:\n')
print('#WE\'VE ALREADY RUN THIS SO DO NOT NEED TO CLEANUP AGAIN\n')

print('gsutil -mq rm -r {}/logs/ubams'.format(COHORT_BUCKET))
print('gsutil -mq rm -r {}/workspace'.format(COHORT_BUCKET))

#### make sure the json dir is there for the next steps

In [ ]:
#create the jsons directory for the ubam to cram if it doesn't exist
#also check to see if the blank jsons are present or you need to retrieve
fastq_to_bam_json_dir = '{}/jsons'.format(WRKDIR)

if os.path.isdir(fastq_to_bam_json_dir):
    os.makedirs(fastq_to_bam_json_dir + '/broadbams', exist_ok=True)    
    
checkfortemplatefile(fastq_to_bam_json_dir + '/blank.align.label.json')
checkfortemplatefile(fastq_to_bam_json_dir + '/PairedEndSingleSampleWf.gatk4.0.options.json')
checkfortemplatefile(fastq_to_bam_json_dir + '/template.broadbam.hg38.json')
checkfortemplatefile(fastq_to_bam_json_dir + '/cromwell_client.py')
checkfortemplatefile(fastq_to_bam_json_dir + '/PairedEndSingleSampleWf.gatk4.0.wdl')

In [ ]:
print('#run these commands at terminal:\n')

print('gsutil cp gs://nihnialng-pd-wgs/tools/broad/blank.align.label.json {}/jsons/'.format(WRKDIR))
print('gsutil cp gs://nihnialng-pd-wgs/tools/broad/PairedEndSingleSampleWf.gatk4.0.options.json \
{}/jsons/'.format(WRKDIR))
print('gsutil cp gs://nihnialng-pd-wgs/tools/broad/template.broadbam.hg38.json {}/jsons/'.format(WRKDIR))
print('gsutil cp gs://nihnialng-pd-wgs/tools/broad/cromwell_client.py {}/jsons/'.format(WRKDIR))
print('gsutil cp gs://nihnialng-pd-wgs/tools/broad/PairedEndSingleSampleWf.gatk4.0.wdl {}/jsons/'.format(WRKDIR))

#### create the ubam to cram per sample json files

In [ ]:
#generate the extra json files for ubams to crams; ie labels and options for alignment wf
from datetime import datetime

json_label_template = '{}/jsons/blank.align.label.json'.format(WRKDIR)
json_options_template = '{}/jsons/PairedEndSingleSampleWf.gatk4.0.options.json'.format(WRKDIR)
json_broad_template = '{}/jsons/template.broadbam.hg38.json'.format(WRKDIR)

sample_ids = cohort_df['oriID'].unique()
for sample_id in sample_ids:
    json_labels_outfile_name = '{}/jsons/broadbams/{}.labels.json'.format(WRKDIR,sample_id)
    json_options_outfile_name = '{}/jsons/broadbams/{}.options.json'.format(WRKDIR,sample_id)
    json_broad_outfile_name = '{}/jsons/broadbams/{}.broadbam.hg38.json'.format(WRKDIR,sample_id)    

    #format and write the label json
    with open(json_label_template) as json_file:  
        label_data = json.load(json_file)
        
        label_data['cohort'] = COHORT.lower()
        label_data['sample'] = sample_id.lower()
        label_data['user'] = MYUSER.lower()
        
        with open(json_labels_outfile_name,'w') as json_outfile:
            json.dump(label_data,json_outfile,sort_keys=True,indent=4)   
    
    #format and write the options json
    with open(json_options_template) as json_file:  
        options_data = json.load(json_file)
        
        options_data['final_workflow_outputs_dir'] = '{}/hg38/align-wf/{}'.format(COHORT_BUCKET, sample_id)
        options_data['final_workflow_log_dir'] = '{}/logs/{}'.format(COHORT_BUCKET, sample_id)
        options_data['final_call_logs_dir'] = '{}/logs/{}'.format(COHORT_BUCKET, sample_id)
        
        with open(json_options_outfile_name,'w') as json_outfile:
            json.dump(options_data,json_outfile,sort_keys=True,indent=4)   

    #format and write the broad json
    get_ubams_cmd = 'gsutil ls {}/ubams/{}/*.unmapped.bam'.format(COHORT_BUCKET,sample_id)
    ubams = !{get_ubams_cmd}

    with open(json_broad_template) as json_file:  
        broad_data = json.load(json_file)
        
        broad_data['PairedEndSingleSampleWorkflow.sample_name'] = sample_id
        broad_data['PairedEndSingleSampleWorkflow.base_file_name'] = sample_id
        broad_data['PairedEndSingleSampleWorkflow.flowcell_unmapped_bams'] = ubams
        broad_data['PairedEndSingleSampleWorkflow.final_gvcf_base_name'] = sample_id
        
        with open(json_broad_outfile_name,'w') as json_outfile:
            json.dump(broad_data,json_outfile,sort_keys=False,indent=4)   

#### setup the cromwell server

In [ ]:
print('#setup a cromwell server for running the alignment wdl jobs\n\
#get Matt\'s cromwell stuff\n\
#already pulled for other projects so copy over from dementia_wgs\n')

print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS, SO DO NOT NEED TO CREATE SERVER AGAIN\n')
print('#cp -r ../dementia_wgs/tools/verily-amp-pd-source {}/tools/'.format(WRKDIR))

print('\n#fire up the cromwell instance')
print('chmod +x {}/tools/verily-amp-pd-source/setup_cromwell_vm/*.sh'.format(WRKDIR))
print('cd {}/tools/verily-amp-pd-source/setup_cromwell_vm/'.format(WRKDIR))
print('./create_cromwell_server.sh {}-cromwell {} n1-highmem-8'\
      .format(COHORT,PROJECT_ID))
print('./configure.sh {}-cromwell {} nihnialng-pd-wgs/{}'\
      .format(COHORT,PROJECT_ID,COHORT.replace('_','-')))

print('\n#When that is up, ssh to the instance:')
print('gcloud --project {} compute ssh {}-cromwell'.format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#And in that SSH session, run:')
print('cd /install')
print('docker-compose -f /install/workspace/config/docker-compose.yml up')


#### create the ssh tunnel to the cromwell server so you can submit jobs

In [ ]:
print('#When cromwell is up, create an SSH tunnel from your workstation:')
print('#run these commands at terminal:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'\
      .format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#after this runs you well actually be logged into the cromwell server, ' \
      'so you will need to open another termincal session on your machine to submit your jobs\n')

In [ ]:
#define format function the ggp cmd
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'echo -n {SAMPLE} OPID=\n\
python {WRKDIR}/jsons/cromwell_client.py \
--wdl {WRKDIR}/jsons/PairedEndSingleSampleWf.gatk4.0.wdl \
--workflow-inputs {WRKDIR}/jsons/broadbams/{SAMPLE}.broadbam.hg38.json \
--workflow-options {WRKDIR}/jsons/broadbams/{SAMPLE}.options.json \
--workflow-labels {WRKDIR}/jsons/broadbams/{SAMPLE}.labels.json'
    return(this_cmd.format(WRKDIR=WRKDIR,COHORT=COHORT,SAMPLE=this_sample))

#iterate over samples formatting the cmds
cohort_bucket = '{}/{}'.format(PRJ_BUCKET,COHORT)
cmds = [formatgcpcmd(sample_id,cohort_bucket) for sample_id in sample_ids]

temp_script_file = '{}/{}.run_ubams_to_crams.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.run_ubams_to_crams.log &'.format(WRKDIR,COHORT.lower()))

In [ ]:
#see how many GCE (Google Compute Engine) instances are running your jobs
PIPELABEL='pairedendsinglesamplewf'

print('#full job worker node count')
!gcloud compute instances list --project {PROJECT_ID} \
--filter "labels:({PIPELABEL} {COHORT} {MYUSER})" | grep RUNNING | wc -l

print('#number of all running instances in project')
!gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l

#### commands for checking statuses using Cromwell REST cmds

In [ ]:
print('#run these commands at terminal:\n')

print('#These actually won\'t be that helpful because you are all using the same server.\n')

print('#When cromwell is up, create an SSH tunnel from your workstation, if not already connected:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'.\
      format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#if tunnel established can check cromwell status\n')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Running"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Submitted"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Failed"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Succeeded"')

#### commands to check progess by counting expected output files

In [ ]:
#check bam, cram, and gvcf counts
print('#These actually won\'t be that helpful because you are all using the same bucket path.\n')
print('#crams')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-ConvertToCram/**.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-ConvertToCram/**.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-GatherBamFiles/**.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-GatherBamFiles/**.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-MergeVCFs/**.g.vcf.gz | wc -l

#### commands for copying the primary output files to a final dest path

In [ ]:
print('#run these commands at terminal:\n')

print('#gvcfs')
print('gsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-MergeVCFs/**.g.vcf.gz* \
{}/hg38/gvcfs/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#crams')
print('gsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-ConvertToCram/**.cram* \
{}/hg38/crams/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#bams')
print('gsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-GatherBamFiles/**.bam \
{}/hg38/bams/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('gsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-GatherBamFiles/**.bai \
{}/hg38/bams/'.format(COHORT_BUCKET,COHORT_BUCKET))

#### confirm counts of the primary output files at final dest path

In [ ]:
#check bam, cram, and gvcf counts at final path
print('#crams')
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.g.vcf.gz | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.g.vcf.gz.tbi | wc -l

#### check that all expected files are there by looking at sample IDs

In [ ]:
%%bash -s "$WRKDIR" "$COHORT_BUCKET" "$COHORT"
#get a list of files that were successfully created
WRKDIR=${1}
COHORT_BUCKET=${2}
COHORT=${3}

gsutil -mq ls ${COHORT_BUCKET}/hg38/crams/*.cram > ${WRKDIR}/${COHORT}.found.files

sed -i s"/gs:\/\/nihnialng-pd-wgs\/${COHORT}\/hg38\/crams\///"g ${WRKDIR}/${COHORT}.found.files
sed -i s"/\///"g ${WRKDIR}/${COHORT}.found.files
sed -i s"/\.cram//"g ${WRKDIR}/${COHORT}.found.files

less ${WRKDIR}/${COHORT}.found.files | wc -l

In [ ]:
#check for any missing expected bams
expected_file = '{}/{}.samples.list'.format(WRKDIR, COHORT)
observed_file = '{}/{}.found.files'.format(WRKDIR, COHORT) 
missing_file = '{}/{}.missing.samples.list'.format(WRKDIR, COHORT)

expected = pd.read_csv(expected_file,header=None)
observed = pd.read_csv(observed_file,header=None)

print(expected.shape)
print(observed.shape)

print(len(set(expected[0]) - set(observed[0])))

missing = expected.loc[~expected[0].isin(observed[0])]
print(missing.shape)
missing.head()

#save the missing list
missing.to_csv(missing_file,header=None,index=None)

#### if all the expected primary output files are present then go ahead and copy of the other output files

In [ ]:
#move the other metrics reports that may be of interest to keep
#need to move all the other summary metric files over to final output
print('#run these commands at terminal:\n')
print('gsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-ValidateCram/**.cram.validation_report ' \
'{}/hg38/align-wf/call-ValidateCram/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp ${}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-CheckContamination/**.preBqsr.selfSM ' \
'{}/hg38/align-wf/call-CheckContamination/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/*/call-GatherBqsrReports/**.recal_data.csv ' \
'{}/hg38/align-wf/call-GatherBqsrReports/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/**_metrics {}/hg38/align-wf/metrics/'.\
      format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/hg38/align-wf/*/PairedEndSingleSampleWorkflow/**.pdf {}/hg38/align-wf/metrics/'.\
      format(COHORT_BUCKET,COHORT_BUCKET))

#### check file counts at destination for primary and other metrics output files

In [ ]:
#check file counts
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.vcf.gz | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.vcf.gz.tbi | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.crai | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bai | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-ValidateCram/*.cram.validation_report | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-CheckContamination/*.preBqsr.selfSM | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-GatherBqsrReports/*.recal_data.csv | wc -l

#### check the cram validation reports

In [ ]:
#pull down the validation reports and check for erros
!mkdir -p {WRKDIR}/temp

!gsutil -mq cp {COHORT_BUCKET}/hg38/align-wf/call-ValidateCram/*.cram.validation_report {WRKDIR}/temp/

!ls {WRKDIR}/temp/*.validation_report | wc -l
!less {WRKDIR}/temp/*.validation_report | grep "No errors found" | wc -l

#clean up the temp validation reports
!rm {WRKDIR}/temp/*.validation_report


# This is where we stopped after alignments

#### do some additional cleanup, I've already run these deletes

including the deletion of the ubams

In [ ]:
#if successfully, clean up log files and workspace path
!echo gsutil -mq rm -r {COHORT_BUCKET}/logs
!echo gsutil -mq rm -r {COHORT_BUCKET}/cromwell-execution/PairedEndSingleSampleWorkflow

In [ ]:
#define format function the gcp cmd to delete additional per sample un-needed files
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'gsutil -mq rm -r {COHORT_BUCKET}/hg38/align-wf/{SAMPLE}/PairedEndSingleSampleWorkflow'
    return(this_cmd.format(COHORT_BUCKET=chrt_bucket,SAMPLE=this_sample))

#here reloading sample_ids
cohort_file_list = '{}/{}.samples.list'.format(WRKDIR,COHORT)
sample_ids = pd.read_csv(cohort_file_list,header=None).to_numpy()
sample_ids = sample_ids.reshape(len(sample_ids))

#iterate over samples formatting the cmds
cmds = [formatgcpcmd(sample_id,COHORT_BUCKET) for sample_id in sample_ids]

temp_script_file = '{}/{}.delete_other_files.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO DELETE AGAIN\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.delete_other_files.log &'.format(WRKDIR,COHORT.lower()))

In [ ]:
#define format function the ggp cmd to delete ubams
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'gsutil -mq rm -r {COHORT_BUCKET}/ubams/{SAMPLE}'
    return(this_cmd.format(COHORT_BUCKET=chrt_bucket,SAMPLE=this_sample))

#iterate over samples formatting the cmds
cmds = [formatgcpcmd(sample_id,COHORT_BUCKET) for sample_id in sample_ids]

temp_script_file = '{}/{}.delete_ubams.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO DELETE AGAIN\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.delete_ubams.log &'.format(WRKDIR,COHORT.lower()))

#### archive the fastqs to nearline storage for some periond of time

In [ ]:
#move fastqs to archvie bucket
fastq_bucket_path = '{}/{}/fastqs/*.fastq.gz'.format(PRJ_BUCKET,COHORT)
archive_bucket_path = 'gs://nihnialng-pd-archive/fastqs/usuhs/{}/'.\
format(COHORTBUILD.replace('.', '_'))

print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO MOVE AGAIN\n')
print('nohup gsutil -mq cp {} {} &'.format(fastq_bucket_path, archive_bucket_path))

In [ ]:
#check that all the files were moved, although should be successful if no error was reported
#should match the original counts from early fastq counting cell
!gsutil ls {archive_bucket_path} | wc -l

#get storage size
!gsutil -mq du -hs {archive_bucket_path}

#### if everything is ok with archiving of fastqs, delete original staging bucket fastqs

In [ ]:
#delete the staging bucket fastqs that USUHS uploaded
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO DELETE AGAIN\n')
print('gsutil -mq rm gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.00/*.fastq.gz')
print('gsutil -mq rm gs://nihnialng-staging-f745d15a/pA3.hernandez.N148.01/*.fastq.gz')
print('gsutil -mq rm {}/fastqs/*.fastq.gz'.format(COHORT_BUCKET))

#### check for any contaminated samples
pull down seq contamination check info

In [ ]:
#pull down the verifybamid check contamination files 
!mkdir -p {WRKDIR}/seqqc

!gsutil -mq cp {COHORT_BUCKET}/hg38/align-wf/call-CheckContamination/*.preBqsr.selfSM {WRKDIR}/seqqc/
!ls {WRKDIR}/seqqc/*.preBqsr.selfSM | wc -l

In [ ]:
%%bash -s "$WRKDIR" "$COHORTBUILD" "$COHORT"
#combine the contamination files into single table
WRKDIR=${1}
COHORTBUILD=${2}
COHORT=${3}

SAMPLESLISTFILE="${WRKDIR}/${COHORT}.samples.list"
METRICSFILE="${WRKDIR}/${COHORTBUILD}.contam.metrics.txt"

if [ -s ${METRICSFILE} ]; then
rm ${METRICSFILE}
fi
echo "id avgdp freemix" > ${METRICSFILE}

ls ${WRKDIR}/seqqc/*.preBqsr.selfSM | xargs -l basename > ${WRKDIR}/selfsm.file.list
sed -i s"/\.preBqsr\.selfSM//"g ${WRKDIR}/selfsm.file.list

while read SAMPLELINE
do
SAMPLE=$(echo ${SAMPLELINE} | awk '{print $1}')
awk -v SAMPLE=${SAMPLE} '$1 == SAMPLE {print $1,$6,$7}' \
${WRKDIR}/seqqc/${SAMPLE}.preBqsr.selfSM >> ${METRICSFILE}
done < ${WRKDIR}/selfsm.file.list

rm ${WRKDIR}/selfsm.file.list

#### check the contamination reports for problems

In [ ]:
#check the contamination and seq coverage rates
contam_metrics_file = '{}/{}.contam.metrics.txt'.format(WRKDIR,COHORTBUILD)

WARN_FREEMIX = 0.02
MAX_FREEMIX = 0.049
MIN_DEPTH = 27

metrics_df = pd.read_csv(contam_metrics_file,sep='\s+')
print(metrics_df.shape)
metrics_df.head()

maybe_contamin_df = metrics_df.loc[(metrics_df['freemix'] <= MAX_FREEMIX) & \
                                   (metrics_df['freemix'] > WARN_FREEMIX)]
print('number of samples maybe contaminated with freemix > {} and < {} is {}'\
      .format(WARN_FREEMIX,MAX_FREEMIX,maybe_contamin_df.shape[0]))
contam_maybe_file = '{}/{}.contamination.possible.samples.txt'.format(WRKDIR,COHORTBUILD)
maybe_contamin_df.to_csv(contam_maybe_file,index=False,sep='\t')

contaminated_df = metrics_df.loc[metrics_df['freemix'] > MAX_FREEMIX]
print('number of samples with freemix > {} is {}'.format(MAX_FREEMIX,contaminated_df.shape[0]))
contam_problems_file = '{}/{}.contaminated.samples.txt'.format(WRKDIR,COHORTBUILD)
contaminated_df.to_csv(contam_problems_file,index=False,sep='\t')

low_cov_df = metrics_df.loc[metrics_df['avgdp'] < MIN_DEPTH]
print('number of samples with avgdp < {} is {}'.format(MIN_DEPTH,low_cov_df.shape[0]))
low_coverage_file = '{}/{}.low_coverage.samples.txt'.format(WRKDIR,COHORTBUILD)
low_cov_df.to_csv(low_coverage_file,index=False,sep='\t')


#### good news there aren't any samples with contamination or low coverage

if there had been, you would want to just exclude the ones with freemix > 5% and decide on excluding the possibly contaminated or low coverage

#### now prep for running the joint genotyping

#### pull down the broad tooling

In [ ]:
#pull down the correct recent Broad tooling
!mkdir -p {WRKDIR}/tools

#don't think wget is easy to install for Mac, maybe use curl instead
# !wget https://raw.githubusercontent.com/gatk-workflows/gatk4-germline-snps-indels/master/joint-discovery-gatk4.wdl \
#     -O {WRKDIR}/tools/joint-discovery-gatk4.wdl
# !wget https://raw.githubusercontent.com/gatk-workflows/gatk4-germline-snps-indels/master/joint-discovery-gatk4.hg38.wgs.inputs.json \
#     -O {WRKDIR}/tools/joint-discovery-gatk4.hg38.wgs.inputs.json

!curl https://raw.githubusercontent.com/gatk-workflows/gatk4-germline-snps-indels/master/joint-discovery-gatk4.wdl \
    -o {WRKDIR}/tools/joint-discovery-gatk4.wdl
!curl https://raw.githubusercontent.com/gatk-workflows/gatk4-germline-snps-indels/master/joint-discovery-gatk4.hg38.wgs.inputs.json \
    -o {WRKDIR}/tools/joint-discovery-gatk4.hg38.wgs.inputs.json
    
!curl https://raw.githubusercontent.com/gatk-workflows/gatk4-germline-snps-indels/master/generic.google-papi.options.json \
    -o {WRKDIR}/tools/generic.google-papi.options.json    

#### update joint calling jsons

In [ ]:
#update some of the json configs for broad joint calling
#generate the extra json files for ubams to crams; ie labels and options for alignment wf

generic_options_template = '{}/tools/generic.google-papi.options.json'.format(WRKDIR)
wdl_input_template = '{}/tools/joint-discovery-gatk4.hg38.wgs.inputs.json'.format(WRKDIR)

json_labels_outfile_name = '{}/jsons/{}.jdgatk4.labels.json'.format(WRKDIR,COHORT)
json_options_outfile_name = '{}/jsons/{}.jdgatk4.options.json'.format(WRKDIR,COHORT)
json_broad_outfile_name = '{}/jsons/{}.jdgatk4.hg38.wgs.inputs.json'.format(WRKDIR,COHORT)    

#format and write the label json
label_data = {}
label_data['workflow'] = 'jdgatk4'
label_data['cohort'] = COHORT.lower()
label_data['user'] = MYUSER.lower()

with open(json_labels_outfile_name,'w') as json_outfile:
    json.dump(label_data,json_outfile,sort_keys=True,indent=4)   

#format and write the options json
with open(generic_options_template) as json_file:  
    options_data = json.load(json_file)
    
    options_data['read_from_cache'] = True
    options_data['write_to_cache'] = True
    options_data['workflow_failure_mode'] = 'ContinueWhilePossible'
    options_data['system.input-read-limits.lines'] = 640000    
    options_data['final_workflow_outputs_dir'] = '{}/hg38/joint_calling'.format(COHORT_BUCKET)
    options_data['final_workflow_log_dir'] = '{}/logs/joint_calling'.format(COHORT_BUCKET)
    options_data['final_call_logs_dir'] = '{}/logs/joint_calling'.format(COHORT_BUCKET)

    with open(json_options_outfile_name,'w') as json_outfile:
        json.dump(options_data,json_outfile,sort_keys=True,indent=4)   

#format and write the broad json
with open(wdl_input_template) as json_file:  
    broad_data = json.load(json_file)
    
    sample_gvcfs_path = '{}/{}.sample.gvcfs.map.txt'.format(COHORT_BUCKET,COHORTBUILD)
    broad_data['JointGenotyping.callset_name'] = COHORTBUILD
    broad_data['JointGenotyping.sample_name_map'] = sample_gvcfs_path

    with open(json_broad_outfile_name,'w') as json_outfile:
        json.dump(broad_data,json_outfile,sort_keys=False,indent=4)   


In [ ]:
%%bash -s "$WRKDIR" "$COHORT_BUCKET" "$COHORT"
#get a list of files that were successfully created
WRKDIR=${1}
COHORT_BUCKET=${2}
COHORT=${3}

gsutil -mq ls ${COHORT_BUCKET}/hg38/crams/*.cram > ${WRKDIR}/${COHORT}.found.files

sed -i s"/gs:\/\/nihnialng-pd-wgs\/${COHORT}\/hg38\/crams\///"g ${WRKDIR}/${COHORT}.found.files
sed -i s"/\///"g ${WRKDIR}/${COHORT}.found.files
sed -i s"/\.cram//"g ${WRKDIR}/${COHORT}.found.files

less ${WRKDIR}/${COHORT}.found.files | wc -l

In [ ]:
#need to create the gvcfs sample map file used as json argument above JointGenotyping.sample_name_map
#instead of just looping expected ids do so from found list in previous cell, allow to move/remove fails

found_file_list = '{}/{}.found.files'.format(WRKDIR,COHORT)
found_ids = pd.read_csv(found_file_list,header=None).to_numpy()
# found_ids = sample_ids.reshape(len(found_ids))
found_ids = found_ids.reshape(len(found_ids))

gvcfs_map_file = '{}/{}.sample.gvcfs.map.txt'.format(WRKDIR,COHORTBUILD)

with open(gvcfs_map_file, 'w') as file_handler:
    for sample_id in found_ids:
        file_handler.write("{}\t{}/hg38/gvcfs/{}.g.vcf.gz\n".\
                           format(sample_id,COHORT_BUCKET,sample_id))

#now copy the map file up to cloud bucket
!gsutil -mq cp {gvcfs_map_file} {COHORT_BUCKET}/


#### now run the joint calling job

In [ ]:
print('#When cromwell is up, create an SSH tunnel from your workstation:')
print('#run these commands at terminal:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'\
      .format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#after this runs you well actually be logged into the cromwell server, ' \
      'so you will need to open another termincal session on your machine to submit your jobs\n')

In [ ]:
#now run the joint calling job

run_cmd = 'python {wrk_dir}/jsons/cromwell_client.py \
--wdl {wrk_dir}/tools/oint-discovery-gatk4.wdl \
--workflow-inputs {wrk_dir}/jsons/{this_cohort}.jdgatk4.hg38.wgs.inputs.json \
--workflow-options {wrk_dir}/jsons/{this_cohort}.jdgatk4.options.json \
--workflow-labels {wrk_dir}/jsons/{this_cohort}.jdgatk4.labels.json'.\
format(wrk_dir=WRKDIR, this_cohort=COHORT)

print('#run these commands at terminal:\n')

print('{} > {}/{}.jdgatk4.jobid'.format(run_cmd, WRKDIR, COHORTBUILD))

# This where we stopped after joint calling

In [ ]:
#see how many GCE (Google Compute Engine) instances are running for joint calling
print('#full job worker node count')
!gcloud compute instances list --project {PROJECT_ID} \
--filter "labels:(jdgatk4 {COHORT} {MYUSER})" | grep RUNNING | wc -l

print('#number of all running instances in project')
!gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l

#### commands for checking statuses using Cromwell REST cmds

In [ ]:
print('#run these commands at terminal:\n')

print('#These actually won\'t be that helpful because you are all using the same server.\n')

print('#When cromwell is up, create an SSH tunnel from your workstation, if not already connected:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'.\
      format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#if tunnel established can check cromwell status\n')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Running"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Submitted"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Failed"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Succeeded"')

## check to make sure expected file(s) are there in this case since < 1K samples will be single vcf

Note that below here would be different for larger cohorts, with file per chromosome

In [ ]:
#see if result file is present
!gsutil ls -lh {COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/*/call-FinalGatherVcf/{COHORTBUILD}.vcf.gz*

#### move file to a final destination path

In [ ]:
#job was run twice to two sets of outputs, grab one and then do some cleanup
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN\n')
in_path = f'{COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/83cca38d-7e42-4ac1-bb7c-278c1343a6d7'
out_path = f'{COHORT_BUCKET}/hg38/JointGenotyping/'
print(f'gsutil -mq cp {in_path }/call-FinalGatherVcf/{COHORTBUILD}.vcf.gz* {out_path}')
print(f'gsutil -mq cp {in_path }/call-CollectMetricsOnFullVcf/* {out_path}')

#### now do some clean up

In [ ]:
#delete logging, cromwell execution path, and detritus from output path
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN\n')

print(f'gsutil -mq rm {in_path }/call-FinalGatherVcf/{COHORTBUILD}.vcf.gz*')
print(f'gsutil -mq rm {in_path }/call-CollectMetricsOnFullVcf/*')
print(f'gsutil -mq rm {in_path }/call-DynamicallyCombineIntervals/*')
print(f'gsutil -mq rm -r {COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/2a88a45a-f6dd-4d61-9bfb-d70d07f04fb7')

print(f'gsutil -mq rm -r {COHORT_BUCKET}/logs')
print(f'gsutil -mq rm -r {COHORT_BUCKET}/cromwell-execution/JointGenotyping')

#### now delete the cromwell server

In [ ]:
#if done with the cromwell server delete it
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN\n')
!echo gcloud compute instances delete {COHORT.replace('_','-')}-cromwell \
--project {PROJECT_ID} --zone us-central1-f

### subset vcfs to just PASSED variants

#### VQSR doesn't really work for the non-PAR sexomes and while you probably won't be doing analsys on sexome variation you will need at lest chrX to perform sex check QC

so submit another vcf subset but only for chrX without PASS filter

In [ ]:
print('#run these commands at terminal:\n')
#prep and run the VQSR subset job
print(f'cut -f 1 {WRKDIR}/{COHORTBUILD}.sample.gvcfs.map.txt > {WRKDIR}/{COHORTBUILD}.samples.list\n')
print(f'gsutil -mq cp {WRKDIR}/{COHORTBUILD}.samples.list {COHORT_BUCKET}/\n')

pipe_yaml = 'gs://nihnialng-pd-wgs/tools/yamls/SubjectSubsetVCF.yaml'

filter_vqsr_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/filtervcf/ \
--inputs INPUTVCF={COHORT_BUCKET}/hg38/JointGenotyping/{COHORTBUILD}.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/hg38/JointGenotyping/{COHORTBUILD}.vcf.gz.tbi \
--inputs SUBJECTLIST={COHORT_BUCKET}/{COHORTBUILD}.samples.list \
--inputs EXCLUDECARROT=\" \" \
--inputs FILTERFLAGS=\"-f PASS --force-samples\" \
--inputs MINAC=\"1\" \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz.tbi \
--labels=pipe=filtervcfs,cohort={COHORT.lower()},user={MYUSER}\n'

print(filter_vqsr_cmd)

chrx_vcf_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/filtervcf/ \
--inputs INPUTVCF={COHORT_BUCKET}/hg38/JointGenotyping/{COHORTBUILD}.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/hg38/JointGenotyping/{COHORTBUILD}.vcf.gz.tbi \
--inputs SUBJECTLIST={COHORT_BUCKET}/{COHORTBUILD}.samples.list \
--inputs EXCLUDECARROT=\" \" \
--inputs FILTERFLAGS=\"--regions chrX --force-samples\" \
--inputs MINAC=\"1\" \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz.tbi \
--labels=pipe=filtervcfs,cohort={COHORT.lower()},chromosome=chrx,user={MYUSER}\n'

print(chrx_vcf_cmd)

#### check status of filter on VQSR pass job using OPID

In [ ]:
#the above google genomics pipeline (ggp) commands should return on OPIDs
#grab to previous operation IDs to use here as opids list
opids = ['EKXH4N7RLRiQuKK7mLHWsX4gpIDW36QGKg9wcm9kdWN0aW9uUXVldWU', \
'EKXH4N7RLRiQuKK7mLHWsX4gpIDW36QGKg9wcm9kdWN0aW9uUXVldWU']

for opid in opids:
    !gcloud alpha genomics operations describe {opid} \
    --format='yaml(done, error, metadata.events)'

#### split multi-allelics, name variants, and thin out vcf verbosity

In [ ]:
#split multi-allelics, name variants, and thin out vcf verbosity
pipe_yaml = 'gs://nihnialng-pd-wgs/tools/yamls/SplitAndNameVCF.yaml'

print('#run these commands at terminal:\n')
split_name_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/splitname/ \
--inputs INPUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz.tbi \
--inputs REFFASTA=gs://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta \
--inputs REFFASTAINDEX=gs://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta.fai \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.gtonly.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.gtonly.vcf.gz.tbi \
--labels=pipe=splitnamevcf,cohort={COHORT.lower()},user={MYUSER}\n'

print(split_name_cmd)

split_name_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/splitname/ \
--inputs INPUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz.tbi \
--inputs REFFASTA=gs://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta \
--inputs REFFASTAINDEX=gs://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta.fai \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.gtonly.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.gtonly.vcf.gz.tbi \
--labels=pipe=splitnamevcf,cohort={COHORT.lower()},user={MYUSER}\n'

print(split_name_cmd)

#### check status on the split multi-allelics, re-IDs, and gtonly job using OPID

In [ ]:
#the above google genomics pipeline (ggp) commands should return on OPIDs
#grab to previous operation IDs to use here as opids list
opids = ['EIaD2unRLRil9uOBusqGsH8gpIDW36QGKg9wcm9kdWN0aW9uUXVldWU', \
'ENfu2unRLRjw7-eD1sqevNEBIKSA1t-kBioPcHJvZHVjdGlvblF1ZXVl']

for opid in opids:
    !gcloud alpha genomics operations describe {opid} \
    --format='yaml(done, error, metadata.events)'

In [ ]:
#confirm output file was successfull created
!gsutil -mq ls -lh {COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.*

In [ ]:
print('#run these commands at terminal:\n')
#clean up log files
print(f'gsutil -mq rm -r {COHORT_BUCKET}/logs/jointcall/filtervcf')

## run QC on samples

you can all run this separately in parallel

#### pull down the genotypes

In [ ]:
!mkdir -p {WRKDIR}/genotypes

!gsutil -mq cp {COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.*gtonly.vcf.gz* {WRKDIR}/genotypes/

#### prior to converting from vcf to plink helps to go ahead and include dx and dx

can format using plink2 sample file psam
#FID IID sex dx

hold over from plink1 sex; male=1, female=2, unknown/other=0

In [ ]:
#cohort psam
sample_psam = f'{WRKDIR}/genotypes/{COHORT}.txt'

#### convert vcfs to plink2

In [ ]:
#convert from vcf to plink2
input_vcf = f'{WRKDIR}/genotypes/{COHORTBUILD}.gtonly.vcf.gz'
out_file_set = f'{WRKDIR}/genotypes/{COHORTBUILD}'

!/Users/mooreank/Downloads/plink2 --vcf {input_vcf} --double-id \
--update-sex /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt col-num=3 \
--pheno /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt --pheno-col-nums 4 \
--silent --allow-extra-chr --make-pgen --out {out_file_set}

input_vcf = f'{WRKDIR}/genotypes/{COHORTBUILD}.chrX.gtonly.vcf.gz'
out_file_set = f'{WRKDIR}/genotypes/{COHORTBUILD}.chrX'

!/Users/mooreank/Downloads/plink2 --vcf {input_vcf} --double-id \
--update-sex /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt col-num=3 \
--pheno /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt --pheno-col-nums 4 \
--silent --allow-extra-chr --make-pgen --out {out_file_set}

In [ ]:
#check creation and logging
!ls {WRKDIR}/genotypes/{COHORTBUILD}.*

!tail {WRKDIR}/genotypes/{COHORTBUILD}.*log

#### trim variants to QC set

In [ ]:
#trim variant to QC set
input_file_set = f'{WRKDIR}/genotypes/{COHORTBUILD}'
out_file_set = f'{WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001'

!mkdir -p {WRKDIR}/qc

!/Users/mooreank/Downloads/plink2 --pfile {input_file_set} \
--geno 0.05 --maf 0.05 --hwe 0.000001 \
--silent --make-bed --out {out_file_set}

input_file_set = f'{WRKDIR}/genotypes/{COHORTBUILD}.chrX'
out_file_set = f'{WRKDIR}/qc/{COHORTBUILD}.chrX.geno05maf05'

!/Users/mooreank/Downloads/plink2 --pfile {input_file_set} \
--geno 0.05 --maf 0.05 \
--silent --make-bed --out {out_file_set}

#### run sexcheck

In [ ]:
%%bash -s "$COHORTBUILD" "$WRKDIR"
#check gender
COHORTBUILD=${1}
WRKDIR=${2}"/qc"

#hg38 non-PAR
awk '$4 > 2800000 && $4 < 155700000 {print $2}' ${WRKDIR}/${COHORTBUILD}.chrX.geno05maf05.bim \
    > ${WRKDIR}/${COHORTBUILD}.chrX.list

plink --bfile ${WRKDIR}/${COHORTBUILD}.chrX.geno05maf05 --extract ${WRKDIR}/${COHORTBUILD}.chrX.list \
--check-sex 0.25 0.75 --silent --out ${WRKDIR}/${COHORTBUILD}.sex

In [ ]:
#check sex test results
sexcheck_df = pd.read_csv(f'{WRKDIR}/qc/{COHORTBUILD}.sex.sexcheck', sep='\s+')
print(f'sexcheck shape is {sexcheck_df.shape}')

fail_sexcheck_df = sexcheck_df.loc[sexcheck_df['STATUS'] == 'PROBLEM']
print(f'number of samples failing sexcheck {fail_sexcheck_df.shape[0]}')
print(fail_sexcheck_df)

fail_sexcheck_df = sexcheck_df.loc[(sexcheck_df['STATUS'] == 'PROBLEM') & \
                                   (sexcheck_df['PEDSEX'] != 0)]
print(f'number of samples failing sexcheck excluding missing info {fail_sexcheck_df.shape}')
print(fail_sexcheck_df)

#### check missingness

In [ ]:
#check missingness
!plink2 --bfile {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001 --missing --autosome \
--silent --out {WRKDIR}/qc/{COHORTBUILD}.missing

In [ ]:
#check missingness results
misstest_df = pd.read_csv(f'{WRKDIR}/qc/{COHORTBUILD}.missing.lmiss', sep='\s+')

#find failed
misstest_failed_df = misstest_df.loc[misstest_df['F_MISS'] > 0.05]

print(f'number samples failing missingness test {misstest_failed_df.shape[0]}')

print(misstest_failed_df)

#### check het rates

In [ ]:
#check het rates
!plink --bfile {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001 --het --autosome \
--silent --out {WRKDIR}/qc/{COHORTBUILD}.het

In [ ]:
#check het rate results

###2 failes

hets_df = pd.read_csv(f'{WRKDIR}/qc/{COHORTBUILD}.het.het', sep='\s+')

#find failed
hets_failed_df = hets_df.loc[(hets_df['F'] > 0.15) | (hets_df['F'] < -0.15)]

print(f'number samples failing heterzygosity check {hets_failed_df.shape[0]}')

print(hets_failed_df)

#### check for relatedness, including duplicates

In [ ]:
# king_related_cmd = f'nohup king --related -b {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001.bed \
# --cpus $(nproc) --degree 2 --prefix {WRKDIR}/qc/{COHORTBUILD}.king.related \
# > {WRKDIR}/qc/{COHORTBUILD}.king.related.log &'

# king_related_cmd = f'nohup /Users/mooreank/Downloads/king --related -b {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001.bed \
#  --degree 2 --prefix {WRKDIR}/qc/{COHORTBUILD}.king.related \
# > {WRKDIR}/qc/{COHORTBUILD}.king.related.log &'


king_related_cmd = f'king --related -b /labshare/anni/from_local/{COHORTBUILD}.geno05maf05hwe000001.bed \
 --degree 2 --prefix /labshare/anni/from_local/{COHORTBUILD}.king.related \
> /labshare/anni/from_local/{COHORTBUILD}.king.related.log &'

print('#run these commands at terminal:\n')
print(king_related_cmd)

In [ ]:
#check the king log
!tail -n 16 {WRKDIR}/qc/{COHORTBUILD}.king.related.log

#### for the duplicates see which ones also failed sexcheck, ie clues how to correctly resolve duplicates

In [ ]:
#get the sex problems
sexcheck = pd.read_csv(f'{WRKDIR}/qc/{COHORTBUILD}.sex.sexcheck',sep='\s+')
sexmismatch = sexcheck.loc[(sexcheck['PEDSEX'] != 0) & (sexcheck['STATUS'] == 'PROBLEM')]

#get the duplicates
rels_df = pd.read_csv(f'{WRKDIR}/qc/{COHORTBUILD}.king.related.kin0',sep='\s+')
print(f'number of related pairs {rels_df.shape[0]}')
print(rels_df['InfType'].value_counts())

#subset just the dups
dups_df = rels_df.loc[rels_df['InfType'] == 'DUP/MZ']

print(f'\njust the DUP/MZ {dups_df.shape[0]}')
print(dups_df['InfType'].value_counts())

print('\nnumber of samples in that failed sexcheck and also a duplicate')
print(len(set(sexmismatch['IID']) & (set(rels_df['ID1']) | set(rels_df['ID2']))))


#### check the sample ancestries making use of 1KG data

In [ ]:
#pull down the 1KG genotypes to look at pop-structure
!gsutil -mq cp gs://nihnialng-pd-wgs/tools/onekg.good.geno05maf05hwe000001.* {WRKDIR}/qc/

In [ ]:
#check the pull downs
!ls -lhtr {WRKDIR}/qc/onekg.*

In [ ]:
%%bash -s "$WRKDIR"
#for our WGS since I renamed all variants by position and allele need to do the same for onekg data
WRKDIR=${1}"/qc"

mv ${WRKDIR}/onekg.good.geno05maf05hwe000001.bim ${WRKDIR}/onekg.good.geno05maf05hwe000001.bim.ori
awk '{ print $1"\tchr"$1":"$4":"$5":"$6"\t"$3"\t"$4"\t"$5"\t"$6}' \
${WRKDIR}/onekg.good.geno05maf05hwe000001.bim.ori > ${WRKDIR}/onekg.good.geno05maf05hwe000001.bim

In [ ]:
%%bash -s "$WRKDIR" "$COHORTBUILD"
#unfortunelty because of the plink2/plink1 ref vs minor freq variant files, \
#so rename the variants to match the flipped alleles
WRKDIR=${1}"/qc"
COHORTBUILD=${2}

mv ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.bim ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.bim.ori
awk '{ print $1"\tchr"$1":"$4":"$5":"$6"\t"$3"\t"$4"\t"$5"\t"$6}' \
${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.bim.ori > ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.bim

In [ ]:
%%bash -s "$COHORTBUILD" "$WRKDIR"
#merge cohort and 1KG data with plink
COHORTBUILD=${1}
WRKDIR=${2}"/qc"

plink --bfile ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001 \
--bmerge ${WRKDIR}/onekg.good.geno05maf05hwe000001 \
    --geno 0.05 --maf 0.05 --hwe 0.000001 --silent --allow-no-sex \
    --keep-allele-order --make-bed --out ${WRKDIR}/${COHORTBUILD}.onekg

if [ -s ${WRKDIR}/${COHORTBUILD}.onekg-merge.missnp ]; then
plink2 -bfile ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001 --allow-no-sex \
    --exclude ${WRKDIR}/${COHORTBUILD}.onekg-merge.missnp --silent \
    --keep-allele-order --make-bed --out ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.temp

plink2 -bfile ${WRKDIR}/onekg.good.geno05maf05hwe000001 --allow-no-sex \
    --exclude ${WRKDIR}/${COHORTBUILD}.onekg-merge.missnp --silent \
    --keep-allele-order --make-bed --out ${WRKDIR}/onekg.good.geno05maf05hwe000001.temp

plink --bfile ${WRKDIR}/${COHORTBUILD}.geno05maf05hwe000001.temp \
    --bmerge ${WRKDIR}/onekg.good.geno05maf05hwe000001.temp \
    --geno 0.05 --maf 0.05 --hwe 0.000001 --silent --allow-no-sex \
    --keep-allele-order --make-bed --out ${WRKDIR}/${COHORTBUILD}.onekg
fi


In [ ]:
%%bash -s "$COHORTBUILD" "$WRKDIR"
#merge cohort and 1KG data with plink
COHORTBUILD=${1}
WRKDIR=${2}"/qc"



In [ ]:
#check the merged cohort and 1KG data with plink was created
!ls -lh {WRKDIR}/qc/{COHORTBUILD}.onekg*

#### pull down the 1KG population labels

In [ ]:
#need to pull down the 1KG population labels
!gsutil -mq cp gs://nihnialng-pd-wgs/tools/onekg.panel {WRKDIR}/qc/
    
#check file
!head {WRKDIR}/qc/onekg.panel

#### king is to slow for population background check once we get into the thousands of samples

for now switch back to LD prunning, plink genome ibs/ibd and mds methods

In [ ]:
%%bash -s "$COHORTBUILD" "$WRKDIR"
#merge cohort and 1KG data with plink
COHORTBUILD=${1}
WRKDIR=${2}"/qc"

echo -e "#run these commands at terminal:\n"

#find pruned variants
echo plink2 --bfile ${WRKDIR}/${COHORTBUILD}.onekg \
--indep-pairwise 50 5 0.3 --out ${WRKDIR}/${COHORTBUILD}.onekg.ldprune 

#subset to pruned variants
echo plink2 --bfile ${WRKDIR}/${COHORTBUILD}.onekg \
--extract ${WRKDIR}/${COHORTBUILD}.onekg.ldprune.prune.in \
--make-bed --out ${WRKDIR}/${COHORTBUILD}.onekg.pruned

#generate ibs/ibd info
echo plink --bfile ${WRKDIR}/${COHORTBUILD}.onekg.pruned \
--genome --out ${WRKDIR}/${COHORTBUILD}.onekg.ibs_ibd

#do mds clusters from ibs/ibd data
echo "nohup plink --bfile ${WRKDIR}/${COHORTBUILD}.onekg.pruned \
--read-genome ${WRKDIR}/${COHORTBUILD}.onekg.ibs_ibd.genome \
--cluster --mds-plot 20 --out ${WRKDIR}/${COHORTBUILD}.onekg.ibs_ibd.mds &"

In [ ]:
qc_dir =  f'{WRKDIR}/qc'
mdsfile = f'{qc_dir}/{COHORTBUILD}.onekg.ibs_ibd.mds.mds'
popinfofile =  f'{qc_dir}/onekg.panel'
output_pca_plot = f'{qc_dir}/{COHORTBUILD}.pop_structure.pca.png'
outfile = f'{qc_dir}/{COHORTBUILD}.pop.outliers.txt'

cohort_label = COHORT

expected_ancestry = 'EUR'

mdsdf = pd.read_csv(mdsfile,sep='\s+')
infodf = pd.read_csv(popinfofile,sep="\t")

df = mdsdf.merge(infodf,how='left',left_on='IID',right_on='sample')

df[['super_pop']] = df[['super_pop']].fillna(cohort_label)
df[['pop']] = df[['pop']].fillna(cohort_label)

ancestry_cohort = df.loc[df['super_pop'] == expected_ancestry]
coi = df.loc[df['super_pop'] == cohort_label]

mds1_mean = np.mean(ancestry_cohort['C1'])
mds1_std = np.std(ancestry_cohort['C1'])
mds2_mean = np.mean(ancestry_cohort['C2'])
mds2_std = np.std(ancestry_cohort['C2'])

maxSD = 6
plusMax_1 = mds1_mean + maxSD * mds1_std
negMax_1 = mds1_mean - maxSD * mds1_std
plusMax_2 = mds2_mean + maxSD * mds2_std
negMax_2 = mds2_mean - maxSD * mds2_std

df['outlier'] = ((df['super_pop'] == cohort_label) & \
    ((df['C1'] > plusMax_1) | (df['C1'] < negMax_1) | \
     (df['C2'] > plusMax_2) | (df['C2'] < negMax_2)))

outliers = df.loc[df['outlier'] == True] 
outliers.to_csv(outfile,sep="\t",index=False,header=False)

df.loc[df['IID'].isin(outliers['IID']),['super_pop','pop']] = 'outlier'

# df = df.sort_values('super_pop',ascending=False)
df = df.sample(frac=1)

cohort_euro_df = df[df['super_pop'].isin([expected_ancestry,cohort_label,'outlier'])]
# cohort_euro_df = cohort_euro_df.sort_values('pop',ascending=False)
# cohort_euro_df = cohort_euro_df.sample(frac=1)

sns.set()
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
sns.scatterplot(x='C1',y='C2',hue='super_pop',style='super_pop',data=df)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc=1, borderaxespad=0,prop={'size': 8})
plt.subplot(1,2,2)
sns.scatterplot(x='C1',y='C2',hue='pop',style='pop',data=cohort_euro_df)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0,prop={'size': 8})
#plt.legend(loc='upper right', prop={'size': 6})
plt.savefig(output_pca_plot,format='png',dpi=600,bbox_inches='tight')
plt.show()

In [ ]:
df.head()

In [ ]:
cohort_euro_df['pop'].value_counts()

In [ ]:
mdsdf.set_index(mdsdf['IID'],inplace=True)
mdsdf.drop(columns=['FID','IID','SOL'],inplace=True)
mdsdf.head()

In [ ]:
#run UMAP on the expression data
from umap import UMAP

umap_results = UMAP(random_state=42).fit_transform(mdsdf)
df_umap = pd.DataFrame(umap_results,columns=['x-umap','y-umap'], \
                                   index=mdsdf.index).round(3)
print(f'The dimensions of the umap df of the population ancestry data are {df_umap.shape}')
df_umap.head()

In [ ]:
df_umap = df_umap.merge(infodf,how='left',left_index=True,right_on='sample')

df_umap[['super_pop']] = df_umap[['super_pop']].fillna(cohort_label)
df_umap[['pop']] = df_umap[['pop']].fillna(cohort_label)

df_umap.loc[df_umap['sample'].isin(outliers['IID']),['super_pop','pop']] = 'outlier'

print(df_umap['super_pop'].value_counts())

cohort_euro_df = df_umap[df_umap['super_pop'].isin([expected_ancestry,cohort_label,'outlier'])]
print(cohort_euro_df['pop'].value_counts())

# cohort_euro_df = cohort_euro_df.sort_values('sample',ascending=False)
cohort_euro_df = cohort_euro_df.sample(frac=1)

In [ ]:
output_umap_plot = f'{qc_dir}/{COHORTBUILD}.pop_structure.umap.png'

sns.set()
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
sns.scatterplot(x='x-umap',y='y-umap',hue='super_pop',style='super_pop',data=df_umap)
plt.xlabel('x-UMAP')
plt.ylabel('y-UMAP')
plt.legend(bbox_to_anchor=(1.05, 1), loc=1, borderaxespad=0,prop={'size': 8})
plt.subplot(1,2,2)
sns.scatterplot(x='x-umap',y='y-umap',hue='pop',data=cohort_euro_df)
plt.xlabel('x-UMAP')
plt.ylabel('y-UMAP')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0,prop={'size': 8})
plt.savefig(output_umap_plot,format='png',dpi=600,bbox_inches='tight')
plt.show()

#### accumulate definite excludes, ie sample QC fails
missingness, sexcheck, het-rate, and contamination

In [ ]:
#sex check mismatches
sexcheck_file = f'{WRKDIR}/qc/{COHORTBUILD}.sex.sexcheck'
sexcheck_df = pd.read_csv(sexcheck_file,sep='\s+')
sexcheck_problems_df = sexcheck_df[(sexcheck_df['STATUS'] == 'PROBLEM') & (sexcheck_df['PEDSEX'] != 0)]
print('sex mismatches')
print(f'{sexcheck_problems_df.shape[0]} failed sexcheck')

#check missingness 
smiss_file = f'{WRKDIR}/qc/{COHORTBUILD}.missing.lmiss'
smiss_df = pd.read_csv(smiss_file,sep='\s+')
smiss_problems_df = smiss_df[smiss_df['F_MISS'] > 0.05]
print(f'{smiss_problems_df.shape[0]} failed missingness check')

#het rate problems
hetcheck_file = f'{WRKDIR}/qc/{COHORTBUILD}.het.het'
hetcheck_df = pd.read_csv(hetcheck_file,sep='\s+')
hetcheck_problems_df = hetcheck_df[(hetcheck_df['F'] > 0.15) | (hetcheck_df['F'] < -0.15)]
print(f'{hetcheck_problems_df.shape[0]} failed het rate check')

#sample contamination problems
# contam_df = pd.DataFrame(data=None,columns=['id'])
contam_file = f'{WRKDIR}/{COHORTBUILD}.contaminated.samples.txt'
contam_df = pd.read_csv(contam_file,sep='\s+')
print(f'{contam_df.shape[0]} failed contamination check')


In [ ]:
# def_exclude_set = set(sexcheck_problems_df['IID']) | set(smiss_problems_df['IID']) | \
# set(hetcheck_problems_df['IID']) | set(contam_df['id']) 

one=set(sexcheck_problems_df['IID'])
two=set(smiss_problems_df['SNP'])
three=set(hetcheck_problems_df['IID'])
four=set(contam_df['id'])

def_exclude_set = one | two | three | four



print(f'{len(def_exclude_set)} samples should be excluded to exclude')